In [ ]:
import pandas as pd
import numpy as np
import os

import ast
import cv2

from sklearn.model_selection import train_test_split
import shutil
from tqdm.notebook import tqdm
import tqdm.notebook as tq

import albumentations as albu
from albumentations import Compose

import matplotlib.pyplot as plt

In [ ]:
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.13.1+cu116 (Tesla T4)


In [ ]:
# clone repo
!git clone https://github.com/ultralytics/yolov5.git  

# change the working directory to yolov5
#%cd yolov5
os.chdir('/content/yolov5')

# install dependencies
%pip install -qr requirements.txt 

# Change the working directory back to /kaggle/working/
os.chdir('/content')

!pwd

Cloning into 'yolov5'...
remote: Enumerating objects: 15356, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15356 (delta 0), reused 0 (delta 0), pack-reused 15355
Receiving objects: 100% (15356/15356), 14.33 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (10503/10503), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.1 MB/s eta 0:00:00
/content


In [ ]:
! pwd

/content


In [ ]:
os.chdir('/content/yolov5')
! pwd

/content/yolov5


In [ ]:
# Create a new directory (this is happening inside the yolov5 directory)

base_dir = 'base_dir'
os.mkdir(base_dir)


# Now we create folders inside 'base_dir':

# base_dir

    # images
        # train
        # validation

    # labels
        # train
        # validation

# images
images = os.path.join(base_dir, 'images')
os.mkdir(images)

# labels
labels = os.path.join(base_dir, 'labels')
os.mkdir(labels)



# Inside each folder we create seperate folders for each class

# create new folders inside images
train = os.path.join(images, 'train')
os.mkdir(train)
validation = os.path.join(images, 'validation')
os.mkdir(validation)


# create new folders inside labels
train = os.path.join(labels, 'train')
os.mkdir(train)
validation = os.path.join(labels, 'validation')
os.mkdir(validation)

In [ ]:
! ls

base_dir       CONTRIBUTING.md	hubconf.py  README.zh-CN.md   train.py
benchmarks.py  data		LICENSE     requirements.txt  tutorial.ipynb
CITATION.cff   detect.py	models	    segment	      utils
classify       export.py	README.md   setup.cfg	      val.py


In [ ]:
import os
import shutil

source_img_train = '/content/drive/MyDrive/training_data/images/train'
destination_img_train = '/content/yolov5/base_dir/images/train'

source_img_val = '/content/drive/MyDrive/training_data/images/val'
destination_img_val = '/content/yolov5/base_dir/images/validation'

source_label_train = '/content/drive/MyDrive/training_data/labels/train'
destination_label_train = '/content/yolov5/base_dir/labels/train'

source_label_val = '/content/drive/MyDrive/training_data/labels/val'
destination_label_val = '/content/yolov5/base_dir/labels/validation'

# gather all files
allfiles_img_train = os.listdir(source_img_train)
allfiles_img_val = os.listdir(source_img_val)
allfiles_label_train = os.listdir(source_label_train)
allfiles_label_val = os.listdir(source_label_val)

# iterate on all files to move them to destination folder
for f in allfiles_img_train:
	src_path_img_train = os.path.join(source_img_train, f)
	dst_path_imag_train = os.path.join(destination_img_train, f)
	shutil.copy(src_path_img_train, dst_path_imag_train)
 
for f in allfiles_img_val:
	src_path_img_val = os.path.join(source_img_val, f)
	dst_path_imag_val = os.path.join(destination_img_val, f)
	shutil.copy(src_path_img_val, dst_path_imag_val)
 
for f in allfiles_label_train:
	src_path_label_train = os.path.join(source_label_train, f)
	dst_path_label_train = os.path.join(destination_label_train, f)
	shutil.copy(src_path_label_train, dst_path_label_train)
 
for f in allfiles_label_val:
	src_path_label_val = os.path.join(source_label_val, f)
	dst_path_label_val = os.path.join(destination_label_val, f)
	shutil.copy(src_path_label_val, dst_path_label_val)


In [ ]:
yaml_dict = {'train': 'base_dir/images/train',   # path to the train folder
            'val': 'base_dir/images/validation', # path to the val folder
            'nc': 4,                             # number of classes
            'names': ['Bad', 'Excellent', 'Good', 'Worse']}                # list of label names

In [ ]:
# Create the yaml file called my_data.yaml
# We will save this file inside the yolov5 folder.

import yaml

with open(r'my_data.yaml', 'w') as file:
    documents = yaml.dump(yaml_dict, file)

In [ ]:
os.listdir('/content/yolov5')

['base_dir',
 'my_data.yaml',
 '.pre-commit-config.yaml',
 '.dockerignore',
 'CONTRIBUTING.md',
 'val.py',
 'README.md',
 'models',
 'hubconf.py',
 '.gitignore',
 'utils',
 'detect.py',
 'CITATION.cff',
 'tutorial.ipynb',
 'setup.cfg',
 '.git',
 'requirements.txt',
 'LICENSE',
 'README.zh-CN.md',
 'data',
 '.github',
 'export.py',
 'train.py',
 'classify',
 'benchmarks.py',
 'segment',
 '.gitattributes']

In [ ]:
# Display the contents of the yaml file

! cat '/content/yolov5/my_data.yaml'

names:
- Bad
- Excellent
- Good
- Worse
nc: 4
train: base_dir/images/train
val: base_dir/images/validation


In [ ]:
! pwd

/content/yolov5


In [ ]:
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 150 --freeze 10 --data my_data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=my_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-132-ga82132c Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

In [ ]:
print(len(os.listdir('/content/yolov5/base_dir/images/train')))
print(len(os.listdir('/content/yolov5/base_dir/images/validation')))

In [ ]:
os.listdir('runs/train/')

In [ ]:
# get a list of experiments
exp_list = os.listdir('runs/train/')

# Get the latest exp.
# I found that the first item in the list is the latest experiment. Not
# the last item as one would normally expect.
exp = exp_list[0]

exp

In [ ]:
# Display the contents of the "exp" folder
os.listdir(f'runs/train/{exp}')

In [ ]:
# One mosaic batch of train images with labels

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread(f'runs/train/{exp}/train_batch0.jpg'))

In [ ]:
# One batch of val images with true labels

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread(f'runs/train/{exp}/val_batch0_labels.jpg'))

In [ ]:
# One batch of val images with predicted labels

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread(f'runs/train/{exp}/val_batch0_pred.jpg'))

In [ ]:
# Make a prediction on the test images

# Note if we had test images, the absolute path to the folder containing the
# test images can be set as follows:
# '/kaggle/input/global-wheat-detection/test'
# (note it's /kaggle/input/ and not /kaggle/working/)

# Adding --save-txt means that after prediction each image will have a txt file with bounding box info.
# https://github.com/ultralytics/yolov5/issues/388
# --save-conf means that we will also save the confidence scores for each bounding box.

# Here we are just making a prediction on images that are inside the 'images' folder.
# This is just a demo. Change the path to point to your test images.
# Note that we are using --save-txt and --save-conf because we want to save the 
# predicted bounding box coordinates and confidence scores.
!python detect.py --source '/content/Good (22).jpg' --weights 'runs/train/exp/weights/best.pt' --img 640 --visualize --save-txt --save-conf --exist-ok

In [ ]:
# Make a prediction on the test images

# Note if we had test images, the absolute path to the folder containing the
# test images can be set as follows:
# '/kaggle/input/global-wheat-detection/test'
# (note it's /kaggle/input/ and not /kaggle/working/)

# Adding --save-txt means that after prediction each image will have a txt file with bounding box info.
# https://github.com/ultralytics/yolov5/issues/388
# --save-conf means that we will also save the confidence scores for each bounding box.

# Here we are just making a prediction on images that are inside the 'images' folder.
# This is just a demo. Change the path to point to your test images.
# Note that we are using --save-txt and --save-conf because we want to save the 
# predicted bounding box coordinates and confidence scores.
!python detect.py --source '/content/Worst (19).jpg' --weights 'runs/train/exp/weights/best.pt' --img 640 --visualize --save-txt --save-conf --exist-ok

In [ ]:
!zip -r /content/detect.zip /content/yolov5/runs/detect
!zip -r /content/train.zip /content/yolov5/runs/train

from google.colab import files
files.download('/content/detect.zip')
files.download('/content/train.zip')

In [ ]:
!python export.py --weights yolov5s.pt --include torchscript

In [ ]:
files.download('/content/yolov5/yolov5s.pt')
files.download('/content/yolov5/yolov5s.torchscript.pt')
files.download('/content/yolov5/yolov5s.torchscript.pt1')

In [ ]:
!zip -r /content/detect1.zip /content/yolov5/runs/detect
files.download('/content/detect1.zip')

In [ ]:
files.download('/content/yolov5/yolov5s.torchscript.pt')

In [ ]:
files.download('/content/yolov5/yolov5s.pt')

In [ ]:
files.download('/content/yolov5/yolov5s.torchscript.pt1')

In [ ]:
files.download('/content/detect1.zip')

In [ ]:
os.chdir('/content')

In [ ]:
!python convert.py